# Group Members
Arnold Kulundu - 104896

Gideon Longishu - 111356

Melvin Ngure - 110497

Adam Boru - 113229

In [49]:
#Imports
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

## The housing dataset upload 

In [50]:
# Uploading the file to the notebook
file = files.upload()

Saving modified_data.csv to modified_data (1).csv


In [51]:
#preparing the dataset for training
data = pd.read_csv("modified_data.csv",header=0,index_col="Id")

In [52]:
#Showimg the da
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [53]:
data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [54]:
# getting the original features in the dataset
original_features = list(data.columns)
len(original_features)

76

In [55]:
#getting the missing values
missing_values=list(data.columns[data.isna().any()])
len(missing_values)

15

In [56]:
# use of bfill and fill to fill in the missing values in the dataset
for column in missing_values:
  if data[column].dtype == 'float64':
    data[column].fillna(data[column].mean(),inplace=True)
  else:
    data[column].fillna(method='bfill',inplace=True)

In [57]:
#use of ffill to to fill in the remaining values
data['FireplaceQu'].fillna(method='ffill',inplace=True)

# Dataset Encoding

In [58]:
#Listing the categorical features in the dataset
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [59]:
# listing the numerical features in the dataset
numerical_features=list(set(original_features)-set(categorical_features))
numerical_features

['YearRemodAdd',
 'BsmtHalfBath',
 'FullBath',
 'BsmtFinSF1',
 'YrSold',
 'TotalBsmtSF',
 'MoSold',
 'MiscVal',
 'SalePrice',
 'MasVnrArea',
 'BsmtUnfSF',
 'GarageYrBlt',
 'BsmtFinSF2',
 'GrLivArea',
 'BedroomAbvGr',
 'Fireplaces',
 'LotArea',
 'BsmtFullBath',
 'ScreenPorch',
 'EnclosedPorch',
 'YearBuilt',
 'LotFrontage',
 'TotRmsAbvGrd',
 '3SsnPorch',
 'OverallQual',
 '2ndFlrSF',
 'GarageArea',
 'PoolArea',
 'WoodDeckSF',
 'HalfBath',
 'OverallCond',
 'KitchenAbvGr',
 'MSSubClass',
 'GarageCars',
 'LowQualFinSF',
 'OpenPorchSF',
 '1stFlrSF']

In [60]:
nominal=['MSZoning','LandContour','Neighborhood','Street', 'LotConfig']
ordinal=list(set(categorical_features)-set(nominal))

In [61]:
target=['SalePrice']

In [62]:
#one-hot encoding the nominal data using get_dummies()
df_nominal = pd.get_dummies(data[nominal])

In [63]:
#encoding ordinal features
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [64]:
# form a new dataset by concatenating the encoded dataframes
new_data=pd.concat([df_nominal,df_ordinal,data[numerical_features]],axis=1)
new_data.shape

(1460, 112)

# Dataset Standardisation

In [65]:

X= new_data.drop('SalePrice',axis=1)

In [66]:
y=data[target]

In [67]:
y=StandardScaler().fit_transform(y)

# Feature selection using L1

In [68]:
regressor=LassoCV()
regressor.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [69]:
coef_col = (regressor.coef_)
accepted_features =(X.columns[(coef_col != 0)])
accepted_features

Index(['YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', 'MiscVal', 'MasVnrArea',
       'GrLivArea', 'LotArea', 'YearBuilt', '2ndFlrSF', 'GarageArea',
       'WoodDeckSF'],
      dtype='object')

In [70]:
rejected_features =  X.columns[(coef_col == 0)]
rejected_features

Index(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low',
       'LandContour_Lvl', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr',
       'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer',
       'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr',
       'Neighborhood_Timber', 'Neighborhood_Veenker', 'Street_Grvl',
       'Street_Pave', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2',
       'LotConfig_FR3', 'LotConfig_Inside', 'LotShape', 'RoofMatl',
       'SaleC

In [71]:
new_X = new_data.drop(rejected_features, axis=1)
new_X = StandardScaler().fit_transform(new_X)

# Training with 70% of the data

In [72]:
train_ratio = 0.7
test_ratio = 0.3
#Splitting the data into test and train data
x_train, x_test, y_train, y_test = train_test_split(new_X, y, test_size=test_ratio)

In [73]:
#Creating the model
epochs =100

model = Sequential()
model.add(Dense(32, input_shape=(new_X.shape[1],), activation='relu'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001),loss="mse", metrics=['MSE'])

history = model.fit(x_train, y_train, verbose=1, epochs=epochs, validation_data=(x_test, y_test))

Epoch 1/100
32/32 [==============================] - 1s 12ms/step - loss: 0.9859 - MSE: 0.9859 - val_loss: 0.5819 - val_MSE: 0.5819
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.5544 - MSE: 0.5544 - val_loss: 0.1927 - val_MSE: 0.1927
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1938 - MSE: 0.1938 - val_loss: 0.1501 - val_MSE: 0.1501
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1762 - MSE: 0.1762 - val_loss: 0.1237 - val_MSE: 0.1237
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1549 - MSE: 0.1549 - val_loss: 0.0924 - val_MSE: 0.0924
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1007 - MSE: 0.1007 - val_loss: 0.0732 - val_MSE: 0.0732
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1155 - MSE: 0.1155 - val_loss: 0.0576 - val_MSE: 0.0576
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0555 - 

# Testing with 30% of the data

In [74]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

In [75]:
train_score = r2_score(y_train, y_train_pred)
train_score

0.9992117039573657

In [76]:
test_score = r2_score(y_test, y_test_pred)
test_score

0.994958148082356